In [ ]:
from workshop_utils import * 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import datetime
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Editors in Tacloban
The following query gets basic editing stats for all objects edited in Tacloban, Philippines, grouped by user

```sql
SELECT changesets.user,
       count(distinct(southeastern_asia.id)) AS objects,
       count(southeastern_asia.id) AS edits,
       count(distinct(southeastern_asia.changeset)) AS changesets,
       min(southeastern_asia.updated) AS first_edit,
       max(southeastern_asia.updated) AS last_edit
FROM southeastern_asia
JOIN changesets
    ON southeastern_asia.changeset = changesets.id
WHERE ST_WITHIN( geom , ST_Polygon('POLYGON((124.909271 11.293189, 125.088518 11.293189, 125.088518 10.954934, 124.909271 10.954934, 124.909271 11.293189))') )
GROUP BY  changesets.user ORDER BY objects DESC
```

In [ ]:
users = load_dataframe_from_s3('https://us-east-2.console.aws.amazon.com/athena/query/results/cc587eaa-78d3-4d0f-b436-b610660f15d4/csv')

In [ ]:
users.head(2)

In [ ]:
# Basic dataframe cleaning and maintenance
users['first_edit'] = users.first_edit.apply(pd.Timestamp)
users['last_edit']  = users.last_edit.apply(pd.Timestamp)

In [ ]:
users['first_day'] = users.first_edit.apply(pd.Timestamp.date)

In [ ]:
ax = users.groupby('first_day').aggregate({'user':'count'}).plot(style='.', figsize=(14,4),legend=None)
ax.set_title("Number of users active in the area",fontsize=16)
ax.set_xlabel("Date"); ax.set_ylabel("Number of Users");
# ax.set_xlim(datetime.date(2012,1,1)); 

In [ ]:
ax = users.groupby('first_day').aggregate({'user':'count'}).cumsum().plot(figsize=(14,4),legend=None)
ax.set_title("Number of users active in the area",fontsize=16)
ax.set_xlabel("Date"); ax.set_ylabel("Number of Users (Cumulative)")
ax.set_xlim(datetime.date(2012,1,1)); 

In [ ]:
ax = users.groupby('first_edit').aggregate({'objects':'sum'}).cumsum().plot(figsize=(14,4))
ax.set_title("Objects edited in Tacloban",fontsize=16);
ax.set_xlabel("Date"); ax.set_ylabel("Object count (cumulative)");

As opposed to just _new buildings_, we can see that all edits in Tacloban follow a similar trend with the most activity happening around Typhoon Haiyan